In [1]:
from IPython.display import HTML
from moduli import oblika, vpetje, obremenitve

oblika_ = []
vpetje_ = []
obremenitve_ = []

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

Avtor: **Jan Tomec, 23140220**, (Blaž Starc)

Datum: 4. januar 2016

*Potrjujem, da sem avtor projektne naloge in da sem vso vsebino pripravil sam. V primeru, da se ugotovi plagiatorstvo se zavedam, da ne bom izpolnjeval pogojev za pristop k izpitu.*

Structural_analysis_v.0.8.2

# Strukturna analiza nosilca

To je v HTML-ju in Javascript-u urejena verzija programa, ki samodejno analizira nosilec, izriše diagram in poišče točke največjega osne in strižne sile ter momenta.

Uvoženi python moduli za računanje so v mapi moduli, slike v mapi html/img, Javascript pa nima posebnih modulov, temveč je kar vključen v HTML.

*To je osnovni program s katerim sem zadovoljil le delu kriterijev iz definicije naloge, zato sem dodal še dodatno datoteko "dodatek.ipynb", kjer sem tudi rešil en statičen problem, pri tem pa sem uporabljal tudi ta program.*

Zaradi pomanjkanja časa in zahtevnosti projekta, je bilo premalo časa za debugging, zato se izjemoma lahko zgodi, da kakšna funkcija ne bo delala, kot bi morala. V splošnem je izračunano pravilno. Ogledate si lahko tudi nekaj že rešenih primerov, ali pa poskusite rešiti svojega.

To je v0.9, dalo bi se dodati še vrsto drugih funkcij.

Opis programa je sproten, dodaten opis pa se lahko najde tudi v posameznih modulih, ki so bogati z docstring-i in komentarji.



## Oblika nosilca

Najprej definirajte obliko nosilca. To storite tako, da vnesete podatke v obliki, kot je nakazano. Za potrditev kliknite ikono pod polji za vpisovanje. Če je nosilec sestavljen iz večih delov, vnesite vsak del posebej.
Pod vpisnim formularjem se izpisujejo podatki, ki ste jih vnesli.
Gumb odstrani zadnji element naredi prav to. Nosilec mora biti povezan in v nobeni točki dvakrat definiran (pri enem x dva različna y).

Vsa ostala vpisna polja delujejo na identičen princip.

Parametri:

Raven nosilec: Povezuje začetno in končno točko z ravnim nosilcem
* start = x , y  -  predstavlja začetno točko nosilca v absolutnem koordinatnem sistemu (ponavadi 0 , 0)
* stop = x , y  -  predstavlja končno točko nosilca v absolutnem koordinatnem sistemu

Krožni nosilec: Naredi krožni lok iz začetne točke okoli središča, dolžina loka se definira s kotom. Opozorilo: funkcijo opazujemo v absolutnem koordinatnem sistemu.
* start = x , y  -  predstavlja začetno točko nosilca v absolutnem koordinatnem sistemu
* center = x , y  -  predstavlja središče krožnice, ki matematično popisuje geometrijo nosilca
* kot = fi  -  predstavlja kot (v stopinjah) krožnega loka; kot se meri v smeri pozitivne x osi; kot naj ne preseže 90 stopinj (lahko pa dodate dve krožna nosilca, ki sestavljata enega večjega)
* predznak = 1  -  relevanten le, ko je središče kroga na isti osi kot začetek (predznak definira, po katerem delu krožnicu bo šel x, 1 (+) predstavlja zgornji del, -1 (-) predstavlja spodnji del

In [2]:
gumbi_oblika = """
<div style="background-color:gainsboro; border:solid black; width:170px; padding:20px;">

<p>Raven nosilec:</p>
<input type="text" id="Raven_start" style="width: 125px" value="start = 0 , 0">
<input type="text" id="Raven_stop" style="width: 125px" value="stop = x , y">
<p><button onclick="set_value_Raven()"><img src="html/img/raven.png" style="height: 60px" /></button></p>

<p>Krožni nosilec:</p>
<input type="text" id="Arc_start" style="width: 125px" value="start = 0 , 0">
<input type="text" id="Arc_center" style="width: 125px" value="center = x , y">
<input type="text" id="Arc_kot" style="width: 125px" value="kot = fi">
<input type="text" id="Arc_predznak" style="width: 125px" value="predznak = 1">
<p><button onclick="set_value_Arc()"><img src="html/img/arc.png" style="height: 60px" /></button></p>

<p><button onclick="oblika_remove()">Odstrani zadnji element</button></p>

</div>
"""

javascript_oblika = """
<script type="text/Javascript">

var Oblika_var = new Array()

    function set_value_Raven(){
        var Raven_start_name = "Raven_start";
        var Raven_start_value = document.getElementById('Raven_start').value;
        var Raven_start_value_split = Raven_start_value.split(" ");
        var Raven_start_x_value = parseFloat(Raven_start_value_split[2]);
        var Raven_start_y_value = parseFloat(Raven_start_value_split[4]);
        
        var Raven_stop_name = "Raven_stop";
        var Raven_stop_value = document.getElementById('Raven_stop').value;
        var Raven_stop_value_split = Raven_stop_value.split(" ");
        var Raven_stop_x_value = parseFloat(Raven_stop_value_split[2]);
        var Raven_stop_y_value = parseFloat(Raven_stop_value_split[4]);
        
        var command0 = Raven_start_name + " = [ " + Raven_start_x_value + " , " + Raven_start_y_value + " ] ";
        var command1 = Raven_stop_name + " = [ " + Raven_stop_x_value + " , " + Raven_stop_y_value + " ] ";
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        
        var command2 = "oblika_.append(oblika.Raven(start = Raven_start, stop = Raven_stop))";
        
        console.log("Executing Command: " + command2);
        
        kernel.execute(command2);
        
        Oblika_var.push([Raven_start_value, Raven_stop_value]);
        document.getElementById("demo_oblika").innerHTML = Oblika_var;
        
    }
    
    function set_value_Arc(){
        
        var Arc_start_name = "Arc_start";
        var Arc_start_value = document.getElementById('Arc_start').value;
        var Arc_start_value_split = Arc_start_value.split(" ");
        var Arc_start_x_value = parseFloat(Arc_start_value_split[2]);
        var Arc_start_y_value = parseFloat(Arc_start_value_split[4]);
        
        var Arc_center_name = "Arc_center";
        var Arc_center_value = document.getElementById('Arc_center').value;
        var Arc_center_value_split = Arc_center_value.split(" ");
        var Arc_center_x_value = parseFloat(Arc_center_value_split[2]);
        var Arc_center_y_value = parseFloat(Arc_center_value_split[4]);
        
        var Arc_kot_name = "Arc_kot";
        var Arc_kot_value = document.getElementById('Arc_kot').value;
        var Arc_kot_value_split = Arc_kot_value.split(" ");
        var Arc_kot_x_value = parseFloat(Arc_kot_value_split[2]);
        
        var Arc_predznak_name = "Arc_predznak";
        var Arc_predznak_value = document.getElementById('Arc_predznak').value;
        var Arc_predznak_value_split = Arc_predznak_value.split(" ");
        var Arc_predznak_x_value = parseFloat(Arc_predznak_value_split[2]);
        
        var command0 = Arc_start_name + " = [ " + Arc_start_x_value + " , " + Arc_start_y_value + " ] ";
        var command1 = Arc_center_name + " = [ " + Arc_center_x_value + " , " + Arc_center_y_value + " ] ";
        var command2 = Arc_kot_name + " = " + Arc_kot_x_value;
        var command3 = Arc_predznak_name + " = " + Arc_predznak_x_value;
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        console.log("Executing Command: " + command2);
        console.log("Executing Command: " + command3);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        kernel.execute(command2);
        kernel.execute(command3);
        
        var command4 = "oblika_.append(oblika.Arc(start = Arc_start, center = Arc_center, kot = Arc_kot, a_predznak = Arc_predznak))";
        
        console.log("Executing Command: " + command4);
        
        kernel.execute(command4);
        
        Oblika_var.push([Arc_start_value, Arc_center_value, Arc_kot_value, Arc_predznak_value]);
        document.getElementById("demo_oblika").innerHTML = Oblika_var;
        
    }
    
    function oblika_remove(){
    
        Oblika_var = Oblika_var.slice(0, -1);
        document.getElementById("demo_oblika").innerHTML = Oblika_var;
        
        var command0 = "oblika_.pop(-1)";
        
        console.log("Executing Command: " + command0);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
    }

</script>"""

izpis_oblika = """<body>
<p id="demo_oblika"></p>
<body/>"""

HTML(gumbi_oblika + javascript_oblika + izpis_oblika)

## Vpetje nosilca

Parametri:

Konzola: Preprečuje rotacijo in translacijo.
* lokacija = x  -  predstavlja x koordinato točke v kateri se nosilec nahaja (y koordinata je določena z obliko)

Nepomična podpora: Preprečuje translacijo.
* lokacija = x  -  predstavlja x koordinato točke v kateri se nosilec nahaja (y koordinata je določena z obliko)

Konzola: Preprečuje translacijo v eni smeri (odvisno od orientacije podpore).
* lokacija = x  -  predstavlja x koordinato točke v kateri se nosilec nahaja (y koordinata je določena z obliko)
* kot = fi  -  predstavlja kot (v stopinjah), za katerega je zarotirana podpora glede na nevtralno os (x os), primer: pri kotu nič je preprečena translacija v y smeri

In [3]:
gumbi_podpore = """
<div style="background-color:gainsboro; border:solid black; width:180px; padding:20px;">

<p>Konzola:</p>
<input type="text" id="Konzola_lokacija" style="width: 135px" value="lokacija = 0">
<p><button onclick="set_value_Konzola()"><img src="html/img/konzola.png" style="height: 60px" /></button></p>

<p>Nepomična podpora:</p>
<input type="text" id="Nepomicna_podpora_lokacija" style="width: 135px" value="lokacija = 0">
<p><button onclick="set_value_Nepomicna_podpora()"><img src="html/img/nepremicna_podpora.png" style="height: 60px" /></button></p>

<p>Pomična podpora:</p>
<input type="text" id="Pomicna_podpora_lokacija" style="width: 135px" value="lokacija = x">
<input type="text" id="Pomicna_podpora_kot" style="width: 135px" value="kot = fi">
<p><button onclick="set_value_Pomicna_podpora()"><img src="html/img/premicna_podpora.png" style="height: 60px" /></button></p>

<p><button onclick="podpore_remove()">Odstrani zadnji element</button></p>

</div>
"""

javascript_podpore = """
<script type="text/Javascript">

var Podpore_var = new Array()

    function set_value_Konzola(){
        var Konzola_lokacija_name = "Konzola_lokacija";
        var Konzola_lokacija_value = document.getElementById('Konzola_lokacija').value;
        var Konzola_lokacija_value_split = Konzola_lokacija_value.split(" ");
        var Konzola_lokacija_x_value = parseFloat(Konzola_lokacija_value_split[2]);
        
        var command0 = Konzola_lokacija_name + " = " + Konzola_lokacija_x_value;
        
        console.log("Executing Command: " + command0);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        
        var command1 = "vpetje_.append(vpetje.Konzola(oblika_, lokacija=Konzola_lokacija))";
        
        console.log("Executing Command: " + command1);
        
        kernel.execute(command1);
        
        Podpore_var.push([Konzola_lokacija_value]);
        document.getElementById("demo_podpore").innerHTML = Podpore_var;
    }
    
    function set_value_Nepomicna_podpora(){
        var Nepomicna_podpora_lokacija_name = "Nepomicna_podpora_lokacija";
        var Nepomicna_podpora_lokacija_value = document.getElementById('Nepomicna_podpora_lokacija').value;
        var Nepomicna_podpora_lokacija_value_split = Nepomicna_podpora_lokacija_value.split(" ");
        var Nepomicna_podpora_lokacija_x_value = parseFloat(Nepomicna_podpora_lokacija_value_split[2]);
        
        var command0 = Nepomicna_podpora_lokacija_name + " = " + Nepomicna_podpora_lokacija_x_value;
        
        console.log("Executing Command: " + command0);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        
        var command1 = "vpetje_.append(vpetje.Nepomična_podpora(oblika_, lokacija=Nepomicna_podpora_lokacija))";
        
        console.log("Executing Command: " + command1);
        
        kernel.execute(command1);
        
        Podpore_var.push([Nepomicna_podpora_lokacija_value]);
        document.getElementById("demo_podpore").innerHTML = Podpore_var;
    }
    
    function set_value_Pomicna_podpora(){
        var Pomicna_podpora_lokacija_name = "Pomicna_podpora_lokacija";
        var Pomicna_podpora_lokacija_value = document.getElementById('Pomicna_podpora_lokacija').value;
        var Pomicna_podpora_lokacija_value_split = Pomicna_podpora_lokacija_value.split(" ");
        var Pomicna_podpora_lokacija_x_value = parseFloat(Pomicna_podpora_lokacija_value_split[2]);
        
        var Pomicna_podpora_kot_name = "Pomicna_podpora_kot";
        var Pomicna_podpora_kot_value = document.getElementById('Pomicna_podpora_kot').value;
        var Pomicna_podpora_kot_value_split = Pomicna_podpora_kot_value.split(" ");
        var Pomicna_podpora_kot_x_value = parseFloat(Pomicna_podpora_kot_value_split[2]);
        
        var command0 = Pomicna_podpora_lokacija_name + " = " + Pomicna_podpora_lokacija_x_value;
        var command1 = Pomicna_podpora_kot_name + " = " + Pomicna_podpora_kot_x_value;
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        
        var command2 = "vpetje_.append(vpetje.Pomična_podpora(oblika_, lokacija=Pomicna_podpora_lokacija))";
        
        console.log("Executing Command: " + command2);
        
        kernel.execute(command2);
        
        Podpore_var.push([Pomicna_podpora_lokacija_value, Pomicna_podpora_kot_value]);
        document.getElementById("demo_podpore").innerHTML = Podpore_var;
    }
    
    function podpore_remove(){
    
        Podpore_var = Podpore_var.slice(0, -1);
        document.getElementById("demo_podpore").innerHTML = Podpore_var;
        
        var command0 = "vpetje_.pop(-1)";
        
        console.log("Executing Command: " + command0);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
    }
    
</script>"""

izpis_podpore = """<body>
<p id="demo_podpore"></p>
<body/>"""

HTML(gumbi_podpore + javascript_podpore + izpis_podpore)

## Zunanje obremenitve

Kontinuirne obremenitve imajo še določene omejitve. Lahko so postavljene samo na vodoravnih nosilcih in delujejo vedno le pravokotno na nosilec. Kontinuirna obremenitev definiramo v koordinatnem sistemu, ki ima x os enako kot absolutni, y os pa ima zamenjano z velikostjo obremenitvije. S tem lahko podamo tudi smer obremenitve.

Parametri:

Točkovna sila:
* start = x  -  predstavlja x koordinato točke v kateri se nahaja prijemališče sile (y koordinata je določena z obliko)
* velikost = F  -  velikost sile
* kot = fi  -  kot zasuka nosilke sile glede na pozitivno x os, primer: kot -90 pomeni, da deluje sila v navpični smeri navzdol

Kontinuirna linearna obremenitev: oblika $q(x) = k \cdot x + n$
* start = x  -  predstavlja x koordinato točke v kateri se nahaja začetek prijemališča kontinuirne obremenitve (y koordinata je določena z obliko)
* stop = x  -  predstavlja x koordinato točke v kateri se nahaja konec prijemališča kontinuirne obremenitve (y koordinata je določena z obliko)
* n = začetna vrednost  -  $n$
* k = naklon  -  $k$

Kontinuirna parabolična obremenitev: oblika $q(x) = a_0 + a_1 \cdot x + a_2 \cdot x^2$
* start = x  -  predstavlja x koordinato točke v kateri se nahaja začetek prijemališča kontinuirne obremenitve (y koordinata je določena z obliko)
* stop = x  -  predstavlja x koordinato točke v kateri se nahaja konec prijemališča kontinuirne obremenitve (y koordinata je določena z obliko)
* koeficient0 = a0  -  $a_0$
* koeficient1 = a1  -  $a_1$
* koeficient2 = a2  -  $a_2$

In [4]:
gumbi_obremenitve = """
<div style="background-color:gainsboro; border:solid black; width:180px; padding:20px;">

<p>Točkovna sila:</p>
<input type="text" id="Tockovna_sila_start" style="width: 135px" value="start = x">
<input type="text" id="Tockovna_sila_velikost" style="width: 135px" value="velikost = -1">
<input type="text" id="Tockovna_sila_kot" style="width: 135px" value="kot = 90">
<p><button onclick="set_value_Tockovna_sila()"><img src="html/img/tockovna_sila.png" style="height: 60px" /></button></p>

<p>Kontinuirna linearna obremenitev:</p>
<input type="text" id="Kontinuirna_linearna_start" style="width: 135px" value="start = x">
<input type="text" id="Kontinuirna_linearna_stop" style="width: 135px" value="stop = x">
<input type="text" id="Kontinuirna_linearna_n" style="width: 135px" value="n = a">
<input type="text" id="Kontinuirna_linearna_k" style="width: 135px" value="k = b">
<p><button onclick="set_value_Kontinuirna_linearna()"><img src="html/img/lin.png" style="height: 60px" /></button></p>

<p>Kontinuirna parabolična obremenitev:</p>
<input type="text" id="Kontinuirna_kvadratna_start" style="width: 135px" value="start = x">
<input type="text" id="Kontinuirna_kvadratna_stop" style="width: 135px" value="stop = x">
<input type="text" id="Kontinuirna_kvadratna_a0" style="width: 135px" value="koeficient0 = a0">
<input type="text" id="Kontinuirna_kvadratna_a1" style="width: 135px" value="koeficient1 = a1">
<input type="text" id="Kontinuirna_kvadratna_a2" style="width: 135px" value="koeficient2 = a2">
<p><button onclick="set_value_Kontinuirna_kvadratna()"><img src="html/img/quad.png" style="height: 60px" /></button></p>

<p><button onclick="obremenitve_remove()">Odstrani zadnji element</button></p>

</div>
"""

javascript_obremenitve = """
<script type="text/Javascript">

var Obremenitve_var = new Array()

    function set_value_Tockovna_sila(){
        var Tockovna_sila_start_name = "Tockovna_sila_start";
        var Tockovna_sila_start_value = document.getElementById('Tockovna_sila_start').value;
        var Tockovna_sila_start_value_split = Tockovna_sila_start_value.split(" ");
        var Tockovna_sila_start_x_value = parseFloat(Tockovna_sila_start_value_split[2]);
        
        var Tockovna_sila_velikost_name = "Tockovna_sila_velikost";
        var Tockovna_sila_velikost_value = document.getElementById('Tockovna_sila_velikost').value;
        var Tockovna_sila_velikost_value_split = Tockovna_sila_velikost_value.split(" ");
        var Tockovna_sila_velikost_x_value = parseFloat(Tockovna_sila_velikost_value_split[2]);
        
        var Tockovna_sila_kot_name = "Tockovna_sila_kot";
        var Tockovna_sila_kot_value = document.getElementById('Tockovna_sila_kot').value;
        var Tockovna_sila_kot_value_split = Tockovna_sila_kot_value.split(" ");
        var Tockovna_sila_kot_x_value = parseFloat(Tockovna_sila_kot_value_split[2]);
        
        var command0 = Tockovna_sila_start_name + " = " + Tockovna_sila_start_x_value;
        var command1 = Tockovna_sila_velikost_name + " = " + Tockovna_sila_velikost_x_value;
        var command2 = Tockovna_sila_kot_name + " = " + Tockovna_sila_kot_x_value;
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        console.log("Executing Command: " + command2);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        kernel.execute(command2);
        
        var command3 = "obremenitve_.append(obremenitve.Točkovna_sila(oblika_, start=Tockovna_sila_start, velikost=Tockovna_sila_velikost, kot=Tockovna_sila_kot))";
        
        console.log("Executing Command: " + command3);
        
        kernel.execute(command3);
        
        Obremenitve_var.push([Tockovna_sila_start_value, Tockovna_sila_velikost_value, Tockovna_sila_kot_value]);
        document.getElementById("demo_obremenitve").innerHTML = Obremenitve_var;
        
    }
    
    function set_value_Kontinuirna_linearna(){
        var Kontinuirna_linearna_start_name = "Kontinuirna_linearna_start";
        var Kontinuirna_linearna_start_value = document.getElementById('Kontinuirna_linearna_start').value;
        var Kontinuirna_linearna_start_value_split = Kontinuirna_linearna_start_value.split(" ");
        var Kontinuirna_linearna_start_x_value = parseFloat(Kontinuirna_linearna_start_value_split[2]);
        
        var Kontinuirna_linearna_stop_name = "Kontinuirna_linearna_stop";
        var Kontinuirna_linearna_stop_value = document.getElementById('Kontinuirna_linearna_stop').value;
        var Kontinuirna_linearna_stop_value_split = Kontinuirna_linearna_stop_value.split(" ");
        var Kontinuirna_linearna_stop_x_value = parseFloat(Kontinuirna_linearna_stop_value_split[2]);
        
        var Kontinuirna_linearna_n_name = "Kontinuirna_linearna_n";
        var Kontinuirna_linearna_n_value = document.getElementById('Kontinuirna_linearna_n').value;
        var Kontinuirna_linearna_n_value_split = Kontinuirna_linearna_n_value.split(" ");
        var Kontinuirna_linearna_n_x_value = parseFloat(Kontinuirna_linearna_n_value_split[2]);
        
        var Kontinuirna_linearna_k_name = "Kontinuirna_linearna_k";
        var Kontinuirna_linearna_k_value = document.getElementById('Kontinuirna_linearna_k').value;
        var Kontinuirna_linearna_k_value_split = Kontinuirna_linearna_k_value.split(" ");
        var Kontinuirna_linearna_k_x_value = parseFloat(Kontinuirna_linearna_k_value_split[2]);
        
        var command0 = Kontinuirna_linearna_start_name + " = " + Kontinuirna_linearna_start_x_value;
        var command1 = Kontinuirna_linearna_stop_name + " = " + Kontinuirna_linearna_stop_x_value;
        var command2 = Kontinuirna_linearna_n_name + " = " + Kontinuirna_linearna_n_x_value;
        var command3 = Kontinuirna_linearna_k_name + " = " + Kontinuirna_linearna_k_x_value;
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        console.log("Executing Command: " + command2);
        console.log("Executing Command: " + command3);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        kernel.execute(command2);
        kernel.execute(command3);
        
        var command4 = "obremenitve_.append(obremenitve.Kontinuirna_linearna(oblika_, start=Kontinuirna_linearna_start, stop=Kontinuirna_linearna_stop, n=Kontinuirna_linearna_n, k=Kontinuirna_linearna_k))";
        
        console.log("Executing Command: " + command4);
        
        kernel.execute(command4);
        
        Obremenitve_var.push([Kontinuirna_linearna_start_value, Kontinuirna_linearna_stop_value, Kontinuirna_linearna_n_value, Kontinuirna_linearna_k_value]);
        document.getElementById("demo_obremenitve").innerHTML = Obremenitve_var;
        
    }
    
    function set_value_Kontinuirna_kvadratna(){
        var Kontinuirna_kvadratna_start_name = "Kontinuirna_kvadratna_start";
        var Kontinuirna_kvadratna_start_value = document.getElementById('Kontinuirna_kvadratna_start').value;
        var Kontinuirna_kvadratna_start_value_split = Kontinuirna_kvadratna_start_value.split(" ");
        var Kontinuirna_kvadratna_start_x_value = parseFloat(Kontinuirna_kvadratna_start_value_split[2]);
        
        var Kontinuirna_kvadratna_stop_name = "Kontinuirna_kvadratna_stop";
        var Kontinuirna_kvadratna_stop_value = document.getElementById('Kontinuirna_kvadratna_stop').value;
        var Kontinuirna_kvadratna_stop_value_split = Kontinuirna_kvadratna_stop_value.split(" ");
        var Kontinuirna_kvadratna_stop_x_value = parseFloat(Kontinuirna_kvadratna_stop_value_split[2]);
        
        var Kontinuirna_kvadratna_a0_name = "Kontinuirna_kvadratna_a0";
        var Kontinuirna_kvadratna_a0_value = document.getElementById('Kontinuirna_kvadratna_a0').value;
        var Kontinuirna_kvadratna_a0_value_split = Kontinuirna_kvadratna_a0_value.split(" ");
        var Kontinuirna_kvadratna_a0_x_value = parseFloat(Kontinuirna_kvadratna_a0_value_split[2]);
        
        var Kontinuirna_kvadratna_a1_name = "Kontinuirna_kvadratna_a1";
        var Kontinuirna_kvadratna_a1_value = document.getElementById('Kontinuirna_kvadratna_a1').value;
        var Kontinuirna_kvadratna_a1_value_split = Kontinuirna_kvadratna_a1_value.split(" ");
        var Kontinuirna_kvadratna_a1_x_value = parseFloat(Kontinuirna_kvadratna_a1_value_split[2]);
        
        var Kontinuirna_kvadratna_a2_name = "Kontinuirna_kvadratna_a2";
        var Kontinuirna_kvadratna_a2_value = document.getElementById('Kontinuirna_kvadratna_a2').value;
        var Kontinuirna_kvadratna_a2_value_split = Kontinuirna_kvadratna_a2_value.split(" ");
        var Kontinuirna_kvadratna_a2_x_value = parseFloat(Kontinuirna_kvadratna_a2_value_split[2]);
        
        var command0 = Kontinuirna_kvadratna_start_name + " = " + Kontinuirna_kvadratna_start_x_value;
        var command1 = Kontinuirna_kvadratna_stop_name + " = " + Kontinuirna_kvadratna_stop_x_value;
        var command2 = Kontinuirna_kvadratna_a0_name + " = " + Kontinuirna_kvadratna_a0_x_value;
        var command3 = Kontinuirna_kvadratna_a1_name + " = " + Kontinuirna_kvadratna_a1_x_value;
        var command4 = Kontinuirna_kvadratna_a2_name + " = " + Kontinuirna_kvadratna_a2_x_value;
        
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        console.log("Executing Command: " + command2);
        console.log("Executing Command: " + command3);
        console.log("Executing Command: " + command4);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        kernel.execute(command2);
        kernel.execute(command3);
        kernel.execute(command4);
        
        var command5 = "obremenitve_.append(obremenitve.Kontinuirna_kvadratna(oblika_, start=Kontinuirna_kvadratna_start, stop=Kontinuirna_kvadratna_stop, a0=Kontinuirna_kvadratna_a0, a1=Kontinuirna_kvadratna_a1, a2=Kontinuirna_kvadratna_a2))";
        
        console.log("Executing Command: " + command5);
        
        kernel.execute(command5);
        
        Obremenitve_var.push([Kontinuirna_kvadratna_start_value, Kontinuirna_kvadratna_stop_value, Kontinuirna_kvadratna_a0_value, Kontinuirna_kvadratna_a1_value, Kontinuirna_kvadratna_a2_value]);
        document.getElementById("demo_obremenitve").innerHTML = Obremenitve_var;
        
    }
    
    function obremenitve_remove(){
    
        Obremenitve_var = Obremenitve_var.slice(0, -1);
        document.getElementById("demo_obremenitve").innerHTML = Obremenitve_var;
        
        var command0 = "obremenitve_.pop(-1)";
        
        console.log("Executing Command: " + command0);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
    }
    
</script>"""

izpis_obremenitve = """<body>
<p id="demo_obremenitve"></p>
<body/>"""

HTML(gumbi_obremenitve + javascript_obremenitve + izpis_obremenitve)

## Izris diagramov

Za izris diagramov notranjih sil in momentov ter skice nosilca pritisnite "Izris diagramov". Za ostale funkcije storite podobno. Funkcija "Shrani sliko" vam bo sliko diagramov shranila v delovno mapo, ime datoteke: "diagrami.png".

In [5]:
from moduli import modul_notranje_sile, izris
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
try:
    a = modul_notranje_sile.NTM(oblika_, obremenitve_, vpetje_)
    b = izris.NTM(oblika_, obremenitve_, vpetje_, a)
    if shrani_bool:
        b.izris(shrani=True, ekstremi=True)
        print("ekstremi osne sile (x): ", b.nicle.Nmax)
        print("ekstremi osne sile (N): ", b.nicle.N_vrednosti_max)
        print("ekstremi osne sile (x): ", b.nicle.Tmax)
        print("ekstremi osne sile (T): ", b.nicle.T_vrednosti_max)
        print("ekstremi osne sile (x): ", b.nicle.Mmax)
        print("ekstremi osne sile (M): ", b.nicle.M_vrednosti_max)
    elif ekstremi_bool:
        b.izris(ekstremi=True)
        print("ekstremi osne sile (x): ", b.nicle.Nmax)
        print("ekstremi osne sile (N): ", b.nicle.N_vrednosti_max)
        print("ekstremi osne sile (x): ", b.nicle.Tmax)
        print("ekstremi osne sile (T): ", b.nicle.T_vrednosti_max)
        print("ekstremi osne sile (x): ", b.nicle.Mmax)
        print("ekstremi osne sile (M): ", b.nicle.M_vrednosti_max)
    else:
        b.izris()
except ValueError:
    print()

input_form = """
<div style="background-color:gainsboro; border:solid black; width:270px; padding:20px;">
<button onclick="izris_NTM()" style="width:110px;">Izris diagramov</button>
<button onclick="nicle_NTM()" style="width:110px;">Določi ekstreme</button>
<button onclick="shrani_NTM()" style="width:110px;">Shrani sliko</button>
<button onclick="shrani_nicle_NTM()" style="width:110px;">Shrani sliko z ekstremi</button>
</div>
"""

javascript = """
<script type="text/Javascript">

    function izris_NTM(){
        var command0 = "ekstremi_bool = False";
        var command1 = "shrani_bool = False";
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        IPython.notebook.execute_cell();
    }

    function shrani_NTM(){
        var command0 = "b.izris(shrani=True, ekstremi=False)";
        console.log("Executing Command: " + command0);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
    }
    
    function nicle_NTM(){
        var command0 = "ekstremi_bool = True";
        var command1 = "shrani_bool = False";
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        IPython.notebook.execute_cell();
        
    }
    
    function shrani_nicle_NTM(){
        var command0 = "ekstremi_bool = True";
        var command1 = "shrani_bool = True";
        console.log("Executing Command: " + command0);
        console.log("Executing Command: " + command1);
        var kernel = IPython.notebook.kernel;
        kernel.execute(command0);
        kernel.execute(command1);
        IPython.notebook.execute_cell();
        
    }
</script>
"""

HTML(input_form + javascript)